In [13]:
from pprint import pprint
import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import requests, json, csv
from twython import Twython, TwythonStreamer
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler, Stream, API, Cursor
import twitter_credentials

In [14]:
#Credentials
consumer_key= twitter_credentials.CONSUMER_SECRET,
consumer_secret=twitter_credentials.CONSUMER_SECRET,
access_token_key=twitter_credentials.ACCESS_TOKEN,
access_token_secret=twitter_credentials.ACCESS_SECRET
    


In [26]:
# Twitter Client
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets

    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets

In [27]:
# Twitter Authenticator
class TwitterAuthenticator():
    def authenticate_twitter_app(self):
        auth = OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token_key,access_token_secret)
        return auth

# Filter out unwanted data
def process_tweet(tweet):
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user'] = tweet['user']['screen_name']
    d['user_loc'] = tweet['user']['location']
    d['long']= tweet['coordinates'][0]
    d['lat']= tweet['coordinates'][1]
    d['place']=tweet['place']

    return d




In [17]:
#Save Tweets to file
class TwitterStreamer():
    def __init__(self):
        self.twitter_authenticator= TwitterAuthenticator()
    
    def stream_tweets(self, fetched_tweets_file, hash_tag_list):
#------------ This handles twitter auth & connection to API ---------------

        listener = TwitterListener(fetched_tweets_file)
    
    #   authenticates twitter app by calling class
        auth=self.twitter_authenticator.authenticate_twitter_app()
   
    # class Stream, passing auth to verify authentication & listener object
        stream = Stream(auth, listener)

    # Stream Filtered Tweets
        stream.filter(track=hash_tag_list)
        
    #save to csv
    def save_to_csv(self, tweet):
        with open(r'saved_tweets.csv', 'a') as file:
            writer = csv.writer(fetched_tweets_file)
            writer.writerow(list(tweet.values()))



In [18]:
class TwitterListener(StreamListener):
#   ------ Listener class just prints out received Tweets --------------  

    def __init__(self, fetched_tweets_file=None):
        if fetched_tweets_file is None:
            fetched_tweets_file={}
        else:
            self.fetched_tweets_file = fetched_tweets_file

    def on_data(self,data):
        try:
            print(data)
            with open(self.fetched_tweets_file,'a') as tf:
                tf.write(data)
                
                return True
        except BaseException as e:
                print("Error on_data %s" % str(e))
        return True
        
    def on_error(self_status):
        print(status)
        
#     Override Errors & print any errors that occur
    def on_error(self,status):
        print(status)

if __name__ == "__main__":
    hash_tag_list = ['bitcoin']
#     changing file to json
    fetched_tweets_file = "twitter.json"
   
    #twitter_streamer = TwitterStreamer()
    #twitter_streamer.stream_tweets(fetched_tweets_file, hash_tag_list)


In [21]:
# ---------------------------------------------------------------------------

# api = twitter_credentials(consumer_key,\
#                           consumer_secret,\
#                           access_token_key,\
#                           access_token_secret)

PRODUCT = "fullarchive"
LABEL = "Development" # This is specific to your application
# i.e. whatever label you set for your Dev environment, and is case sensitive
# The following should be uncommented for the first web request.
# This code was run in a Python notebook, in separate cells.
# It might need to be restructured for other uses.
list_tweets = []
next = None
web_request_count = 0

In [25]:
while next is not None:
    r = api.request("tweets/search/%s/:%s" % (PRODUCT, LABEL),        
               {"query":"point_radius:[-85.6602 30.1588 25mi]",
                "toDate":"201810180000",
                "fromDate":"201810010000",
#                 "next": next # This has to be commented out for the first web request
                }) # This is for Panama City, Florida
    print("r.status_code: ", r.status_code)
    next = r.json()["next"]
    print("web_request_count: ", web_request_count)
    web_request_count += 1
    results = r.json()["results"]
    
    for tweet in results:
        coordinates = tweet['coordinates']['coordinates']
        tweet_date = tweet['created_at']
        tweet_text = tweet['text']
        
        if 'extended_tweet' in tweet.keys():
            tweet_text = tweet['extended_tweet']['full_text']
            tweet_row = {'long_laT':coordinates,
                        'date_utc':tweet_date,
                        'full_text':tweet_text}
            list_tweets.append(tweet_row)
            df = pd.DataFrame(list_tweets)
                
    df.to_json('tweets_df_panama_city_25mi_oct.json', orient='records')
    time.sleep(2.1) # wait i.e. only 30 requests per minute allowed
    
